In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json

import common
from search_tools import create_search_tools

evidently_search_tools = create_search_tools()

In [2]:
import inspect

search_tool = {
    "type": "function",
    "function": {
        "name": evidently_search_tools.search.__name__,
        "description": inspect.getdoc(evidently_search_tools.search),
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to look up in the index"
                }
            },
            "required": ["query"]
        }
    }
}

get_file_tool = {
    "type": "function",
    "function": {
        "name": evidently_search_tools.get_file.__name__,
        "description": inspect.getdoc(evidently_search_tools.get_file),
        "parameters": {
            "type": "object",
            "properties": {
                "filename": {
                    "type": "string",
                    "description": "The filename to retrieve (e.g., 'docs/setup/installation.mdx')"
                }
            },
            "required": ["filename"]
        }
    }
}

In [3]:
tool_pairs = [
    (evidently_search_tools.search, search_tool),
    (evidently_search_tools.get_file, get_file_tool)
]

In [4]:
tools = [t for (_, t) in tool_pairs]
tools_index = {t['function']['name']: f for (f, t) in tool_pairs}

In [6]:
def make_call(tool_call):
    function = tool_call.function
    name = function.name

    if name in tools_index:
        func = tools_index[name]
        arguments = json.loads(function.arguments)
        result = func(**arguments)
    else:
        result = f"Tool '{name}' not found in tools index"

    return {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(result)
    }

In [7]:
from dataclasses import dataclass

@dataclass
class Function:
    name: str
    arguments: str

@dataclass
class ToolCall:
    id: str
    function: Function
    type: str

tool_call = ToolCall(
    id='call_qKib5siFCJlGHGL2yim6g69l',
    function=Function(
        arguments='{"query":"create a dashboard"}',
        name='search'
    ),
    type='function'
)

tool_call_result = make_call(tool_call)
tool_call_result

{'role': 'tool',
 'tool_call_id': 'call_qKib5siFCJlGHGL2yim6g69l',
 'content': '[{"title": "Add dashboard panels (UI)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["**Dashboards** let you **create** Panels to...", "**Dashboards** let you **create** Panels to visualize eval...", "...ject.\\n\\n<Check>\\n  No-code **Dashboards** are available in the Evi..."], "total_matches": 14}, "filename": "docs/platform/dashboard_add_panels_ui.mdx"}, {"title": "Add dashboard panels (API)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["...oard_add_panels_ui).\\n\\n## **Dashboard** Management\\n\\n<Check>\\n  **Dashboards**...", "...rd Management\\n\\n<Check>\\n  **Dashboards** as code are available in...", "...(/docs/setup/cloud) and [**create** a Project](/docs/platfor..."], "total_matches": 32}, "filename": "docs/platform/dashboard_add_panels.mdx"}, {"title": "LLM as a judge", "description": "How to create a

In [5]:
from openai import OpenAI
openai_client = OpenAI()

In [9]:
messages = [
    {"role": "system", "content": common.SINGLE_TOOL_SYSTEM_PROMPT},
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

# 1st call - initial

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    tools=tools,
)

In [10]:
message = response.choices[0].message
message.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_Js8aRhzHUdLFM9L49Aq77pmb', function=Function(arguments='{"query":"create dashboard"}', name='search'), type='function')]

In [11]:
tool_call = message.tool_calls[0]
tool_call

ChatCompletionMessageFunctionToolCall(id='call_Js8aRhzHUdLFM9L49Aq77pmb', function=Function(arguments='{"query":"create dashboard"}', name='search'), type='function')

In [12]:
tool_call_result = make_call(tool_call)

In [13]:
messages.append(message)
messages.append(tool_call_result)

In [14]:
# 2nd call

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    tools=tools,
)

In [15]:
message = response.choices[0].message
message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_zMzrfMFKZBQD3XqAIqdmT7cM', function=Function(arguments='{"filename": "docs/platform/dashboard_overview.mdx"}', name='get_file'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_r7kESvicoGVA7qOZfn1APjlb', function=Function(arguments='{"filename": "docs/platform/dashboard_add_panels_ui.mdx"}', name='get_file'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_NLD9FvBJoVC6faneHlHMizql', function=Function(arguments='{"filename": "docs/platform/dashboard_add_panels.mdx"}', name='get_file'), type='function')])

In [16]:
messages.append(message)

In [17]:
message.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_zMzrfMFKZBQD3XqAIqdmT7cM', function=Function(arguments='{"filename": "docs/platform/dashboard_overview.mdx"}', name='get_file'), type='function'),
 ChatCompletionMessageFunctionToolCall(id='call_r7kESvicoGVA7qOZfn1APjlb', function=Function(arguments='{"filename": "docs/platform/dashboard_add_panels_ui.mdx"}', name='get_file'), type='function'),
 ChatCompletionMessageFunctionToolCall(id='call_NLD9FvBJoVC6faneHlHMizql', function=Function(arguments='{"filename": "docs/platform/dashboard_add_panels.mdx"}', name='get_file'), type='function')]

In [18]:
if message.content:
    print('ASSISTANT:', message.content)

has_function_calls = (message.tool_calls is not None) and \
    (len(message.tool_calls) > 0)

if has_function_calls:
    for tool_call in message.tool_calls:
        function = tool_call.function
        print(f'executing {function.name}({function.arguments})...')
        tool_call_output = make_call(tool_call)
        messages.append(tool_call_output)

executing get_file({"filename": "docs/platform/dashboard_overview.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_add_panels_ui.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_add_panels.mdx"})...


In [19]:
# 3rd call - final

response = openai_client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    tools=tools,
)

In [20]:
message = response.choices[0].message

In [21]:
print(message.content)

To create a dashboard in Evidently, you can follow these steps:

1. **Understanding the Dashboard**: A Dashboard provides a clear view of your AI application's performance, allowing you to track evaluation results and live production quality over time. Every project has its own Dashboard which is initially empty until you run evaluations and save reports.

2. **Creating a Dashboard**:
   - You can add panels to your Dashboard using either the user interface (available in Cloud and Enterprise versions) or through the Python API.
   - To get started, make sure to run an evaluation and save at least one report to your project, as the Dashboard relies on these reports for visualization.

3. **Adding Tabs and Panels**:
   - You can organize your panels into multiple tabs. To add a new tab:
     1. Enter "Edit" mode on the Dashboard.
     2. Click the plus sign to add a Tab and enter a name.
   - To add panels:
     1. In "Edit" mode, click on "Add Panel."
     2. Configure the panel by sele

In [22]:
message_history = [
    {"role": "system", "content": common.SINGLE_TOOL_SYSTEM_PROMPT},
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

iteration_number = 1

while True:
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message_history,
        tools=tools,
    )

    print(f'iteration number {iteration_number}...')
    message = response.choices[0].message
    message_history.append(message)

    if message.content:
        print('ASSISTANT:', message.content)
    
    has_function_calls = (message.tool_calls is not None) and \
        (len(message.tool_calls) > 0)
    
    if has_function_calls:
        for tool_call in message.tool_calls:
            function = tool_call.function
            print(f'executing {function.name}({function.arguments})...')
            tool_call_output = make_call(tool_call)
            message_history.append(tool_call_output)

    if not has_function_calls:
        break

    iteration_number = iteration_number + 1
    print()

iteration number 1...
executing search({"query":"create a dashboard"})...

iteration number 2...
executing get_file({"filename": "docs/platform/dashboard_add_panels_ui.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_add_panels.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_overview.mdx"})...

iteration number 3...
ASSISTANT: To create a dashboard, you can use either the user interface or the Python API. Here’s a brief overview of both methods:

### Using the User Interface
1. **Access Dashboard**: Go to the dashboard section in your project.
2. **Add Tabs**: 
   - Enter "Edit" mode on the Dashboard.
   - Click the plus sign that says "Add Tab" on the left.
   - Optionally, create a custom Tab or select from pre-built Tabs.
3. **Add Panels**:
   - While in "Edit" mode, click on the "Add Panel" button.
   - Configure the panel by selecting the metric, filtering by tag or metric label if necessary.
   - Set the panel type (e.g., pie chart, line plot)

In [6]:
from agent_chat_completions import OpenAIChatCompletionsAgent

In [7]:
agent = OpenAIChatCompletionsAgent(
    llm_client=OpenAI(),
    model='gpt-4o-mini',
    instructions=common.SINGLE_TOOL_SYSTEM_PROMPT,
    tool_pairs=tool_pairs
)

In [8]:
messages = agent.loop(common.DEFAULT_QUESTION)

iteration number 1...
executing search({"query":"create a dashboard"})...

iteration number 2...
executing get_file({"filename": "docs/platform/dashboard_overview.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_add_panels_ui.mdx"})...
executing get_file({"filename": "docs/platform/dashboard_add_panels.mdx"})...

iteration number 3...
ASSISTANT: To create a dashboard, you have two main approaches: using the user interface (UI) or the Python API. Here are the steps for both methods:

### Using the User Interface (UI)
1. **Access the Dashboard**: Each Project has its own Dashboard, which starts empty. 
2. **Run an Evaluation**: Before you can populate the Dashboard with data, you need to run an evaluation and save at least one Report within the Project.
3. **Add Tabs (Optional)**:
   - Enter "Edit" mode on the Dashboard by clicking the mode switch in the top right corner.
   - Click the plus sign to add a new Tab; you can create a blank Tab or select from pre-built temp